In [20]:
import pandas as pd
import numpy as np
import gensim.downloader as api
import seaborn as sns
from tqdm.notebook import tqdm

In [21]:
glove_model = api.load('glove-wiki-gigaword-300')
w2v_model = api.load('word2vec-google-news-300')
word_list = '/Users/jessica_1/Workspace/Wordle_project/wordle-nyt-answers-alphabetical.txt'

# Function to calculate cosine similarity between two vectors
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Function to compute distance between two words using GloVe
def glove_distance(word1, word2, model):
    if word1 in model and word2 in model:
        vec1 = model[word1]
        vec2 = model[word2]
        
        # Calculate Cosine Similarity
        similarity = cosine_similarity(vec1, vec2)
        distance = 1 - similarity  # Cosine distance

        return similarity, distance  # Return if both words are found
    else:
        # Return None for similarity and distance if words are not found
        return None, None

In [ ]:
word_list = '/Users/jessica_1/Workspace/Wordle_project/wordle-nyt-answers-alphabetical.txt'

with open(word_list, 'r') as f:
    words = f.read().splitlines()

test_words = words

df_results = pd.DataFrame(columns=['Word1', 'Word2', 'Similarity', 'Distance'])

total_comparisons = (len(test_words) * (len(test_words) - 1)) // 2  # Total number of comparisons
progress_bar = tqdm(total=total_comparisons, desc="Comparing words", unit="comparison", leave=True, mininterval=0.1, ncols=100)

for i in range(len(test_words)):
    for j in range(i + 1, len(test_words)):  
        word1 = test_words[i]
        word2 = test_words[j]
        similarity, distance = glove_distance(word1, word2, glove_model)

        if similarity is not None: 
            new_row = pd.DataFrame({
                'Word1': [word1],
                'Word2': [word2],
                'Similarity': [similarity],
                'Distance': [distance]
            })
            # Use pd.concat() to append the new row to df_results
            df_results = pd.concat([df_results, new_row], ignore_index=True)
        else:
            print(f"One or both words ('{word1}', '{word2}') not found in the GloVe model.")

        progress_bar.update(1)  # Update progress bar after each comparison

progress_bar.close()  # Close the progress bar when done

In [ ]:
glove_avg_similarity = df_results['Similarity'].mean()
glove_std_similarity = df_results['Similarity'].std()

glove_avg_distance = df_results['Distance'].mean()
glove_std_distance = df_results['Distance'].std()

print(f"output dataframe: {df_results.shape}")
print(f"total possible combination of words: {total_comparisons}")
print(f"number of available pairs: {df_results.shape[0]}")
print(f"glove average similarity: {glove_avg_similarity}")
print(f"glove similarity std dev: {glove_std_similarity}")

print(f"glove average distance: {glove_avg_distance}")
print(f"glove distance std dev: {glove_std_distance}")

# Display the DataFrame to verify
print(df_results.head())

In [29]:
import matplotlib.pyplot as plt


In [ ]:
plt.figure(figsize=(12, 6))

# Plot the similarity distribution
plt.subplot(1, 2, 1)  # Create subplots
sns.histplot(df_results['Similarity'], kde=True, bins=30)
plt.title('Similarity Distribution')
plt.xlabel('Similarity')
plt.ylabel('Count')

# Plot the distance distribution
plt.subplot(1, 2, 2)
sns.histplot(df_results['Distance'], kde=True, bins=30)
plt.title('Distance Distribution')
plt.xlabel('Distance')
plt.ylabel('Count')

# Show the plots
plt.tight_layout()  # Adjusts layout to prevent overlapping
plt.show()

In [24]:
glove_model = api.load('glove-wiki-gigaword-300')
w2v_model = api.load('word2vec-google-news-300')
word_list = '/Users/jessica_1/Workspace/Wordle_project/wordle-nyt-answers-alphabetical.txt'

# Function to calculate cosine similarity between two vectors
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Function to compute distance between two words using GloVe
def word2vec_distance(word1, word2, model):
    if word1 in model and word2 in model:
        vec1 = model[word1]
        vec2 = model[word2]
        
        # Calculate Cosine Similarity
        similarity = cosine_similarity(vec1, vec2)
        distance = 1 - similarity  # Cosine distance
        
        return similarity, distance
    else:
        return None, None 

---

In [ ]:
word_list = '/Users/jessica_1/Workspace/Wordle_project/wordle-nyt-answers-alphabetical.txt'

with open(word_list, 'r') as f:
    words = f.read().splitlines()

test_words = words

w2v_results = pd.DataFrame(columns=['Word1', 'Word2', 'Similarity', 'Distance'])

total_comparisons = (len(test_words) * (len(test_words) - 1)) // 2  # Total number of comparisons
progress_bar = tqdm(total=total_comparisons, desc="Comparing words", unit="comparison", leave=True, mininterval=0.1, ncols=100)

for i in range(len(test_words)):
    for j in range(i + 1, len(test_words)):  
        word1 = test_words[i]
        word2 = test_words[j]
        similarity, distance = glove_distance(word1, word2, w2v_model)

        if similarity is not None: 
            new_row = pd.DataFrame({
                'Word1': [word1],
                'Word2': [word2],
                'Similarity': [similarity],
                'Distance': [distance]
            })
            # Use pd.concat() to append the new row to w2v_results
            w2v_results = pd.concat([w2v_results, new_row], ignore_index=True)
        else:
            print(f"One or both words ('{word1}', '{word2}') not found in the w2v model.")

        progress_bar.update(1)  # Update progress bar after each comparison

progress_bar.close()  # Close the progress bar when done

In [ ]:
w2v_avg_similarity = w2v_results['Similarity'].mean()
w2v_std_similarity = w2v_results['Similarity'].std()

w2v_avg_distance = w2v_results['Distance'].mean()
w2v_std_distance = w2v_results['Distance'].std()

print(f"output dataframe: {w2v_results.shape}")
print(f"total possible combination of words: {total_comparisons}")
print(f"number of available pairs: {w2v_results.shape[0]}")
print(f"w2v average similarity: {w2v_avg_similarity}")
print(f"w2v similarity std dev: {w2v_std_similarity}")

print(f"w2v average distance: {w2v_avg_distance}")
print(f"w2v distance std dev: {w2v_std_distance}")

# Display the DataFrame to verify
print(w2v_results.head())

In [ ]:
plt.figure(figsize=(12, 6))

# Plot the similarity distribution
plt.subplot(1, 2, 1)  # Create subplots
sns.histplot(w2v_results['Similarity'], kde=True, bins=30)
plt.title('w2v Similarity Distribution')
plt.xlabel('Similarity')
plt.ylabel('Count')

# Plot the distance distribution
plt.subplot(1, 2, 2)
sns.histplot(w2v_results['Distance'], kde=True, bins=30)
plt.title('w2v Distance Distribution')
plt.xlabel('Distance')
plt.ylabel('Count')

# Show the plots
plt.tight_layout()  # Adjusts layout to prevent overlapping
plt.show()

In [32]:
df_results.to_pickle('/Users/jessica_1/Workspace/Wordle_project/glove_distance_similarity.pkl')
w2v_results.to_pickle('/Users/jessica_1/Workspace/Wordle_project/w2v_distance_similarity.pkl')


In [33]:
test_w2v = pd.read_pickle('/Users/jessica_1/Workspace/Wordle_project/w2v_distance_similarity.pkl')
test_glove = pd.read_pickle('/Users/jessica_1/Workspace/Wordle_project/glove_distance_similarity.pkl')

In [ ]:
print(test_w2v.head())
print(test_glove.head())
